In [1]:
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split

def split_and_save_data(input_dir, output_dir, test_size=0.2, random_state=42):
    """
    Split data into training and test sets and save to respective directories.

    Parameters:
    - input_dir (str): Path to the input data directory.
    - output_dir (str): Path to the output directory where the split data will be saved.
    - test_size (float): Proportion of the dataset to include in the test split (default is 0.2).
    - random_state (int): Seed for the random number generator (default is 42).
    """

    # Create output directories
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')

    for directory in [train_dir, test_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    # Get the list of class directories
    class_directories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for class_directory in class_directories:
        class_input_path = os.path.join(input_dir, class_directory)
        class_train_path = os.path.join(train_dir, class_directory)
        class_test_path = os.path.join(test_dir, class_directory)

        # Create class directories in train and test
        for directory in [class_train_path, class_test_path]:
            if not os.path.exists(directory):
                os.makedirs(directory)

        # Get the list of image files in the class directory
        image_files = [f for f in os.listdir(class_input_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

        # Split the data into training and test sets
        train_images, test_images = train_test_split(image_files, test_size=test_size, random_state=random_state)

        # Copy images to the respective directories
        for image in train_images:
            copyfile(os.path.join(class_input_path, image), os.path.join(class_train_path, image))

        for image in test_images:
            copyfile(os.path.join(class_input_path, image), os.path.join(class_test_path, image))

# Replace these paths with your actual data paths
input_data_directory = '/kaggle/input/indian-food-classification/Food Classification'
output_split_directory = '/kaggle/working/'

split_and_save_data(input_data_directory, output_split_directory)
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-food-classification/Food Classification/momos/208.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/029.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/014.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/275.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/212.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/239.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/150.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/109.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/034.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/149.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/187.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/024.jpg
/kaggle/input/indian-food-classification/Food Classification/momos/033.jpg
/kaggle/input/indian-food

In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Check if GPU is available and TensorFlow is using it
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Define paths
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

# Image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and augment test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Build ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the layers of the pre-trained model

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),  # Increase the number of neurons
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Adjust the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 50  # Increase the number of epochs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

# Save the model
model.save('/kaggle/working/food_model_resnet50.h5')

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


2024-02-22 10:20:13.874809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 10:20:13.874908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 10:20:14.039909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2 Physical GPUs, 2 Logical GPUs
Found 4997 images belonging to 20 classes.
Found 1259 images belonging to 20 classes.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/50


I0000 00:00:1708597240.785983      75 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 [==============================] - 148s 884ms/step - loss: 2.1984 - accuracy: 0.4569 - val_loss: 0.9181 - val_accuracy: 0.7252
Epoch 2/50
157/157 [==============================] - 135s 860ms/step - loss: 1.1281 - accuracy: 0.6832 - val_loss: 0.7511 - val_accuracy: 0.7832
Epoch 3/50
157/157 [==============================] - 135s 861ms/step - loss: 0.8789 - accuracy: 0.7454 - val_loss: 0.7370 - val_accuracy: 0.7959
Epoch 4/50
157/157 [==============================] - 134s 850ms/step - loss: 0.7267 - accuracy: 0.7777 - val_loss: 0.6687 - val_accuracy: 0.8086
Epoch 5/50
157/157 [==============================] - 133s 846ms/step - loss: 0.6077 - accuracy: 0.8191 - val_loss: 0.6755 - val_accuracy: 0.8205
Epoch 6/50
157/157 [==============================] - 134s 850ms/step - loss: 0.5317 - accuracy: 0.8453 - val_loss: 0.6471 - val_accuracy: 0.8253
Epoch 7/50
157/157 [==============================] - 134s 854ms/step - loss: 0.4597 - accuracy: 0.8595 - val_loss: 0.6945 - val_accura

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 18s 445ms/step - loss: 1.0812 - accuracy: 0.8491
Test Accuracy: 84.91%
